<font color = 'blue'> ***IMPORTING ALL THE NECESSARY LIBRARIES***(readme step 1) </font>

In [ ]:
import pandas as pd
import numpy as np
import urllib.request as request
from skimage import io
import sys
import requests
from io import BytesIO
from PIL import Image
from skimage.color import rgb2gray
from pandas import ExcelWriter
from pandas import ExcelFile
import re
from datetime import date 
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import heapq
import html

<font color = 'blue'> **IMPORTING THE DATAFILES** (readme step 2) </font>

In [ ]:
data = pd.read_csv('steam.csv')
data2 = pd.read_csv('steam_media_data.csv')
data3 = pd.read_csv('steam_description_data.csv')

<font color = 'blue'> ***PRE PROCESSING STEPS***</font>

<font color = 'blue'> Creating empty columns in dataframe (readme step 3)</font>

In [ ]:
data['DLC'] =0 
data['Action'] = 0
data['In_app_purchase'] = 0
data['Windows'] = 0
data['MAC'] = 0
data['Linux'] = 0
data['total_ratings'] = 0
data2['contrast_mean']=0.0
data2["contrast_stdev"]=0.0
data2["contrast_median"]=0.0
data2["brightness"]=0.0

<font color = 'blue'> creating total rating and positive rating percentage columsin the dataframe (readme step 4)</font>

In [ ]:
data['total_ratings']= data['positive_ratings'] + data['negative_ratings']
data['positive_percent']= data['positive_ratings']/data['total_ratings']

<font color = 'blue'> analysing reviews and owners (readme step 5) </font>

In [ ]:
datt2 = data.groupby('owners').mean()['total_ratings']

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
datt2.sort_values()

<font color = 'blue'> Converting oweners as a continuous variable Y (readme step 6)</font>

In [ ]:
for i, row in data.iterrows():
    data.at[i,'lower_owners'] = int(row['owners'].split('-')[0])
    data.at[i,'upper_owners'] = int(row['owners'].split('-')[1])
df_max = pd.DataFrame(data.groupby('owners').max()['total_ratings'])
df_min = pd.DataFrame(data.groupby('owners').min()['total_ratings'])
df_count = pd.DataFrame(data.groupby('owners').count()['total_ratings'])
data['y'] = 0
for i, row in data.iterrows():
    if(df_min.loc[row['owners']]['total_ratings'] == row['total_ratings']):
        data.at[i,'y'] = row['lower_owners']
    elif(df_max.loc[row['owners']]['total_ratings'] == row['total_ratings']):
        data.at[i,'y'] = row['upper_owners']
    else:
        max_rev = df_max.loc[row['owners']]['total_ratings']
        min_rev = df_min.loc[row['owners']]['total_ratings']
        step = (row['upper_owners']-row['lower_owners'])/(max_rev - min_rev)
        data.at[i,'y'] = row['lower_owners'] + step*(row['total_ratings'] - min_rev)

<font color = 'blue'>**SCRAPING** the data to find the Downloadable content presence in free game if any (readme step 7)</font>

<font color = 'blue'> using the url of games available in the data file, we scrap whether the game has downloadable content or not </font>

In [ ]:
for i in range(len(datasetmod['appid'])):
    
    if data['price'][i] != 0.00:
        data['DLC'][i] = 'NA'
        
    else:
        
        inapp = []
        
        from selenium import webdriver
        path="chromedriver.exe"

        import time
        import random
        from selenium import webdriver
        from selenium.webdriver.chrome.options import Options
        chrome_options = Options()
        #chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        
        url = 'https://store.steampowered.com/app/' + str(dataset1['appid'][i]) + '/'
        driver = webdriver.Chrome(path, options = chrome_options)
        driver.get(url)

        driver.title

        # assert 'Team Fortress 2 on Steam' in driver.title
        time.sleep(random.uniform(2,5))

        # Scraping for dlc      

        p1 = driver.find_elements_by_xpath('//*[@id="game_area_purchase"]/h2')

        for hamilton in p1:
            inapp.append(hamilton.text)
            
        if inapp == []:
            data['DLC'][i] = 0
            
        if inapp != []:
            
            data['DLC'][i] = 1
        driver.close()

<font color = 'blue'> Creating dummy variables for genres, In App Purchase and Platforms (readme step 8)</font>

In [ ]:
for i in range(len(data['appid'])):
    
    if 'Action' in data['genres'][i].split(';'):
        data['Action'][i] = 1
        
    if 'In-App Purchases' in data['categories'][i].split(';'):
        data['In_app_purchase'][i] = 1
    
    y = data['platforms'][i].split(';')
    
    if "windows" in y:
        data['Windows'][i] = 1

    if "mac" in y:
        data['MAC'][i] = 1
        
    if "linux" in y:
        data['Linux'][i] = 1

<font color = 'blue'> **IMAGE PROCESSING** to generate brightness and contrast variables for game images (readme step 9)</font>

<font color = 'blue'> Reading all the image urls from the data2 file</font>

In [ ]:
for i in range(len(data2["header_image"])):
    start = data2["header_image"][i].find("?")
    rem = data2["header_image"][i][:start]
    data2["header_image"][i] = rem
def load_image(url):
    image_stream = request.urlopen(url)
    return io.imread(image_stream, plugin='pil')

<font color = 'blue'> Calculating the contrast of every image in the data2 file</font>

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage import io
for i in range(len(data2["header_image"])):
    img = io.imread(data2["header_image"][i],0)
    hist,bins = np.histogram(img.flatten(),256,[0,256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max()/ cdf.max()
    cdf_m = np.ma.masked_equal(cdf,0)
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
    cdf = np.ma.filled(cdf_m,0).astype('uint8')
    stdev = np.std(cdf_m)
    median = np.median(cdf_m)
    mean = np.mean(cdf_m)
    data2['contrast_mean'][i] = mean
    data2["contrast_stdev"][i]=stdev
    data2["contrast_median"][i]=median
    

<font color = 'blue'> Calculating the brightness of every image in the data2 file</font>

In [ ]:
def calculate_brightness(image):
    greyscale_image = image.convert('L')
    histogram = greyscale_image.histogram()
    pixels = sum(histogram)
    brightness = scale = len(histogram)

    for index in range(0, scale):
        ratio = histogram[index] / pixels
        brightness += ratio * (-scale + index)

    return 1 if brightness == 255 else brightness / scale
for i in range(len(data2["header_image"])):
    try:
        response = requests.get(data2['header_image'][i])
        image = Image.open(BytesIO(response.content))
        data2["brightness"][i]=calculate_brightness(image)
    except OSError:
        
        continue


<font color = 'blue'>merging the two dataset using left join on data </font>

In [ ]:
data = pd.merge(data,data2, left_on='appid', right_on='steam_appid' )

<font color = 'blue'> removing useless columns</font>

In [ ]:
data = data[data['english']==1]
data = data.drop(['steam_api','header_image','screenshots','background','movies','total_ratings','lower_owners','upper_owners','categories','genres','steamspy_tags', 'positive_ratings','negative_ratings','average_playtime','platforms'],axis = 1)

<font color = 'blue'> Sentimental analysis and **WORD CLOUD** (readme step 10)</font>

In [ ]:
data1 = pd.read_csv('steam.csv')
dataa = pd.merge(data1,data3,left_on = 'appid', right_on = 'steam_appid')
lines = []
for i in range(len(dataa['appid'])):
    
    y = dataa['owners'][i].split('-')[0]  
    if int(y) >= 100000:
        
        lines.append(dataa['detailed_description'][i])

In [ ]:
import re
def striphtml(dat):
    p = re.compile(r'<.*?>')
    return p.sub('', dat)

line_3=[]
for j in range(len(lines)):
    line_3.append(str.strip(striphtml(lines[j])))

In [ ]:
ff= ''
for i in range(len(line_3)):
    line_3[i] = line_3[i].replace('quot','')
    ff = ff+ line_3[i]

In [ ]:
from wordcloud import WordCloud,STOPWORDS
# Start with one review:
text = ff

new_stopwords = STOPWORDS


# Create and generate a word cloud image:
wordcloud = WordCloud(max_font_size=1000, max_words=50, background_color="white",width=800, height=400, stopwords=new_stopwords, normalize_plurals= True).generate(text)
# wordcloud.recolor(color_func = grey_color_func)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('word2.pdf')

<font color = 'blue'> **NATURAL LANGUAGE PROCESSING** to find the popular topics in the description of games using **NLTK** on data3 file (readme step 11)</font>

<font color = 'blue'> **TOKENIZATION** and stop word removal</font>

In [ ]:
data3a['detailed_description'][1298]

In [ ]:
nltk.download('stopwords')

In [ ]:
data3a = pd.merge(data1,data3,left_on = 'appid', right_on = 'steam_appid')

data3a = data3a[data3a['english'] == 1]

lines = []
for i,row in data3a.iterrows():
    
    lines.append(row['detailed_description'])

import re
def striphtml(dat):
    p = re.compile(r'<.*?>')
    return p.sub('', dat)

line_3=[]
for j in range(len(lines)):
    line_3.append(str.strip(striphtml(lines[j])))

#TOKENIZE

token = []

for j in range(len(line_3)):
    token.append(nltk.word_tokenize(line_3[j]))

#remove stop words
    
from nltk.corpus import stopwords

stop_words_removed=[]

for m in range(len(token)):
    stop_words_removed.append([t1 for t1 in token[m] if not t1 in stopwords.words('english') if t1.isalpha()])

print(stop_words_removed)        
                 


<font color = 'blue'> Lemmatization </font>

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
lemm=[]

for k in range(len(stop_words_removed)):
    lemm.append([lemmatizer.lemmatize(t) for t in stop_words_removed[k] if t.isalpha()])
             
    
# reviews in Count vectors

from sklearn.feature_extraction.text import CountVectorizer

stop=[]
for j in range(len(lemm)):
    s=" ".join(stop_words_removed[j])
    stop.append(s)

vectorizer2 = CountVectorizer(ngram_range=(1,2),min_df =5,max_df=1000)
vectorizer2.fit(stop)
print(vectorizer2.vocabulary_)
v1 = vectorizer2.transform(stop)
print(v1.toarray())


# reviews in Term Frequency-Inverse Document Frequency Vectors

from sklearn.feature_extraction.text import TfidfVectorizer

stop=[]
for j in range(len(stop_words_removed)):
    s=" ".join(stop_words_removed[j])
    stop.append(s)


vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df =5,max_df=1000)
vectorizer.fit(stop)
print(vectorizer.vocabulary_)
v = vectorizer.transform(stop)
print(v.toarray())

<font color = 'blue'> **Latent Dirichlet allocation** model </font>

In [ ]:
# LDA Model-Fit

terms = vectorizer2.get_feature_names()
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=6, max_iter=20, learning_method='online', learning_offset=50.,random_state=0).fit(v1)
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx))
    print(" ".join([terms[i] for i in topic.argsort()[:-4-1:-1]]))
    

# LDA Model - Transformation
topics=[]
for i in range(len(df_2)):
    topics.append(lda.transform(v1[i])[0])

topic_dist = lda.transform(v1)

df_topic=pd.DataFrame(topic_dist)

df_topic.to_csv('Topics.csv', header=True, index=False) 